## Aula 04 - Aplicação


### Módulo Offline

In [44]:
import pandas as pd
import numpy as np
import logging
from tqdm.autonotebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix
from datetime import datetime
from common import get_conn, load_winex_ratings 
from common import Settings, settings_simple, settings_winex

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
logger = logging.getLogger('Calculador de similaridades item-item')


In [8]:

def normalize(x:pd.Series):
    #print(x.dtype)
    #print(x)
    x = x.astype(float)
    x_sum = x.sum()
    #Pego o número de valores não nulos
    x_num = x.astype(bool).sum()
    x_mean = x_sum / x_num
    #Formas de evitar a fração não ter o denominador igual a zero
    if x_num == 1 or x.std() == 0:
        return 0.0
    return (x - x_mean) / (x.max() - x.min())

def normalize_canonical(y:pd.Series):
    x = y.astype(float).to_numpy()
    x_sum = x.sum()
    #Pego o número de valores não nulos
    x_num = x.astype(bool).sum()
    x_mean = x_sum / x_num
    #Formas de evitar a fração não ter o denominador igual a zero
    if x_num == 1 or x.std() == 0:
        return 0.0
    x[x.nonzero()] -=  x_mean
    return x

In [45]:
# Implementing our cosine similarity
def calc_adjusted_cos_sim_nozero(normalized_ratings_item_a, normalized_ratings_item_b):
    '''Cálculo da similaridade porém levando em consideração apenas os pares onde houve avaliação. 
    '''
    #Primeiro os itens onde é zero (nao há avaliação) não entram na conta.
    # Assim, trabalhamos apenas com os valores não zerados.
    
    non_zero_indexes_of_item_a = set(np.flatnonzero(normalized_ratings_item_a))
    non_zero_indexes_of_item_b = set(np.flatnonzero(normalized_ratings_item_b))
    non_zero_indexes = list(non_zero_indexes_of_item_a & non_zero_indexes_of_item_b)
    #print(f" Selected indexes: {non_zero_indexes}")
    normalized_ratings_item_a = normalized_ratings_item_a[non_zero_indexes]
    normalized_ratings_item_b = normalized_ratings_item_b[non_zero_indexes]
    
    
    den_part_a = np.sqrt(sum(np.square(normalized_ratings_item_a)))
    #print(f"Vector of item a: {normalized_ratings_item_a}")
    #print(f"Vector of item b: {normalized_ratings_item_b}")
    num = sum(np.multiply(normalized_ratings_item_a,normalized_ratings_item_b))
    den_part_b = np.sqrt(sum(np.square(normalized_ratings_item_b)))
    #print(f"Formula: {num}/({den_part_a}.{den_part_b})")
    den = den_part_a*den_part_b
    sim_a_b = 0.0 if den==0.0 else num/den
    return sim_a_b

def cosine_sim_nonnull(coo:coo_matrix, **kargs):
    dense_output=kargs.get('dense_output',True)
    M = coo.toarray()
    Sim = np.zeros((M.shape[0], M.shape[0]))
    
    for i in tqdm(range(len(coo.row))):
        item = coo.row[i]
        row_item = M[item, :]
        for item2 in coo.row:
            if item == item2:
                Sim[item, item2] = 1.0
            else:
                Sim[item, item2] = calc_adjusted_cos_sim_nozero(row_item, M[item2, :])
    
    if dense_output == False:
        Sim = coo_matrix(Sim)
    return Sim

            

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

class ItemSimilarityMatrixBuilder:
    def __init__(self, settings:Settings, **kargs):
        self.settings = settings
        self.min_overlap = kargs.get('min_overlap', 15)
        self.min_sim = kargs.get('min_sim', 0.2)
        
    @staticmethod
    def normalize(x:np.array):
        x = x.astype(float)
        x_sum = x.sum()
        #Pego o número de valores não nulos
        x_num = x.astype(bool).sum()
        x_mean = x_sum / x_num
        #Formas de evitar a fração não ter o denominador igual a zero
        if x_num == 1 or x.std() == 0:
            return 0.0
        return (x - x_mean) / (x.max() - x.min())

    def build(self, ratings: pd.DataFrame, **kargs):
        save = kargs.get('save', False)
        norm_function = kargs.get('norm_function', ItemSimilarityMatrixBuilder.normalize)
        logger.debug(f"Função de normalização: {norm_function.__name__}")
        sim_function = kargs.get('sim_function', cosine_similarity)
        logger.debug(f"Função de similaridade: {sim_function.__name__}")

        user_column = self.settings.user_column
        item_column = self.settings.item_column
        rating_column = self.settings.rating_column        
        logger.debug(f"Calculando similaridades ... usando {len(ratings)} avaliações")
        
        start_time = datetime.now()
        logger.debug("Criando a matriz de avaliações")
        #Certificando que este campo é de fato Float
        ratings[rating_column] = ratings[rating_column].astype(float)
        #Cria a coluna avg com a Rating normalizada conforme estudamos
        ratings['avg'] = ratings.groupby(user_column)[rating_column].transform(lambda x: norm_function(x))
        #Crtifica que este campo é de fato Float
        ratings['avg'] = ratings['avg'].astype(float)
        #Transforma os campos UserID e WineID em categorias para que possam ser usados como índices na matriz esparsa
        ratings[user_column] = ratings[user_column].astype('category')
        ratings[item_column] = ratings[item_column].astype('category')
        #Convertemos as avaliações em uma matriz esparsa
        coo = coo_matrix((ratings['avg'].astype(float),
                          (ratings[item_column].cat.codes.copy(),
                           ratings[user_column].cat.codes.copy())))

        logger.debug("Calculando as avaliações sobrepostas entre os itens")
        overlap_matrix = coo.astype(bool).astype(int) \
                            .dot(coo.transpose().astype(bool).astype(int))

        #Calcula a matriz de overlap de avaliações entre os vinhos
        number_of_overlaps = (overlap_matrix > self.min_overlap).count_nonzero()
        number_of_nonzero = overlap_matrix.count_nonzero()
        logger.debug(f"A matriz de overlap deixa {number_of_overlaps} de {number_of_nonzero} com min_overlap={self.min_overlap}")
        logger.debug("Finalizado processo de montagem da matriz de avaliação")
        end_time = datetime.now() - start_time
        logger.debug(f"Matriz({coo.shape[0]}x{coo.shape[1]}) em {end_time} segundos")
        sparsity_level = 1 - (ratings.shape[0] / (coo.shape[0] * coo.shape[1]))
        logger.debug(f"Nível de dispersão da matriz: {sparsity_level}")

        start_time = datetime.now()
        #Calcula a similaridade de cosseno entre as linhas
        #o parâmetro dense_output=False indica que a matriz resultante também deve ser esparsa
        cor = sim_function(coo, dense_output=False)
        #Remove as similaridades que são menores do que o limiar definido
        cor = cor.multiply(cor > self.min_sim)
        # Remove similaridades que não possuem o mínimo de overlap definido
        cor = cor.multiply(overlap_matrix > self.min_overlap)
        #Cria um dicionário dos WinesIds para encontrarmos os elementos a serem buscados posteriormente
        items = dict(enumerate(ratings[item_column].cat.categories))
        end_time = datetime.now() - start_time
        logger.debug(f'Similaridade calculada com sucesso {end_time} segundos')
        if save:
            start_time = datetime.now()
            logger.debug('Iniciando o processo gravação dos dados')
            self._save_similarities(cor, items)
            end_time = datetime.now() - start_time
            logger.debug(f'Processo de gravação finalizado em {end_time} segundos')

        return cor, items

    def _save_similarities(self, sm, index, created=datetime.now()):
        start_time = datetime.now()
        sims = []
        no_saved = 0
        start_time = datetime.now()
        coo = coo_matrix(sm)
        csr = coo.tocsr()
        end_time = datetime.now() - start_time
        logger.debug(f'Instanciação da coo_matrix realizada em {end_time} seconds')

        query = "insert into similarity (created, source, target, similarity) values (?,?,?,?)"

        conn = get_conn(self.settings)
        cur = conn.cursor()
        cur.execute("DROP TABLE IF EXISTS similarity;")
        cur.execute(f'''
            CREATE TABLE IF NOT EXISTS similarity (
                created TEXT,
                source INTEGER,
                target INTEGER,
                similarity REAL
            )
        ''')
        logger.debug(f'Há {coo.count_nonzero()} similaridades a serem salvas')
        xs, ys = coo.nonzero()
        for x, y in tqdm(zip(xs, ys), leave=True):
            if x == y:
                continue
            sim = csr[x, y]
            if sim < self.min_sim:
                continue
            #Salva em blocos de 500000 itens
            if len(sims) == 500000:
                cur.executemany(query, sims)
                sims = []
                logger.debug("{} saved in {}".format(no_saved,
                                                     datetime.now() - start_time))

            new_similarity = (str(created), index[x], index[y], sim)
            no_saved += 1
            sims.append(new_similarity)

        if len(sims)>0:
            cur.executemany(query, sims)
        conn.commit()
        conn.close()
        end_time = datetime.now() - start_time
        logger.debug(f'Foram salvos {no_saved} itens de similaridade. Demorou {end_time} segundos')


#### Teste com o Simple2.csv

In [47]:
# Testar com o simple
# Removi os itens com 0 para que passem saiam da lista de avaliados
df = pd.read_csv('./simple2.csv')
options= {'min_overlap': 2, 
          'min_sim': 0.01, 
          'norm_function': normalize_canonical,
          'sim_function': cosine_sim_nonnull,
          'save': True
          }
builder = ItemSimilarityMatrixBuilder(settings_simple, **options)
matrix, items = builder.build(df, **options)
print(matrix.toarray())

2023-12-20 17:30:34,437 : DEBUG : Função de normalização: normalize_canonical
2023-12-20 17:30:34,438 : DEBUG : Função de similaridade: cosine_sim_nonnull
2023-12-20 17:30:34,440 : DEBUG : Calculando similaridades ... usando 33 avaliações
2023-12-20 17:30:34,441 : DEBUG : Criando a matriz de avaliações
2023-12-20 17:30:34,475 : DEBUG : Calculando as avaliações sobrepostas entre os itens
2023-12-20 17:30:34,483 : DEBUG : A matriz de overlap deixa 36 de 36 com min_overlap=2
2023-12-20 17:30:34,484 : DEBUG : Finalizado processo de montagem da matriz de avaliação
2023-12-20 17:30:34,486 : DEBUG : Matriz(6x6) em 0:00:00.044999 segundos
2023-12-20 17:30:34,489 : DEBUG : Nível de dispersão da matriz: 0.08333333333333337
100%|██████████| 33/33 [00:00<00:00, 507.73it/s]
2023-12-20 17:30:34,572 : DEBUG : Similaridade calculada com sucesso 0:00:00.078998 segundos
2023-12-20 17:30:34,573 : DEBUG : Iniciando o processo gravação dos dados
2023-12-20 17:30:34,575 : DEBUG : Instanciação da coo_matrix 

[[1.         0.         1.         0.         0.         0.        ]
 [0.         1.         0.01744555 0.         0.57949565 0.        ]
 [1.         0.01744555 1.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.06901774 0.95840592]
 [0.         0.57949565 0.         0.06901774 1.         0.        ]
 [0.         0.         0.         0.95840592 0.         1.        ]]


#### Teste com a winex

In [48]:
# Testar com o winex
df = load_winex_ratings()
options= {'min_overlap': 5, 
          'min_sim': 0.02, 
          #'norm_function': normalize_canonical,
          #'sim_function': cosine_sim_nonnull,
          'save': True
          }
builder = ItemSimilarityMatrixBuilder(settings_winex, **options)
matrix, items = builder.build(df, **options)
print(matrix.toarray())

2023-12-20 17:30:50,085 : DEBUG : Função de normalização: normalize
2023-12-20 17:30:50,087 : DEBUG : Função de similaridade: cosine_similarity
2023-12-20 17:30:50,088 : DEBUG : Calculando similaridades ... usando 149753 avaliações
2023-12-20 17:30:50,089 : DEBUG : Criando a matriz de avaliações
2023-12-20 17:31:12,878 : DEBUG : Calculando as avaliações sobrepostas entre os itens
2023-12-20 17:31:13,124 : DEBUG : A matriz de overlap deixa 98310 de 373862 com min_overlap=5
2023-12-20 17:31:13,125 : DEBUG : Finalizado processo de montagem da matriz de avaliação
2023-12-20 17:31:13,126 : DEBUG : Matriz(1003x10314) em 0:00:23.037395 segundos
2023-12-20 17:31:13,127 : DEBUG : Nível de dispersão da matriz: 0.98552403677082
2023-12-20 17:31:13,203 : DEBUG : Similaridade calculada com sucesso 0:00:00.075052 segundos
2023-12-20 17:31:13,205 : DEBUG : Iniciando o processo gravação dos dados
2023-12-20 17:31:13,208 : DEBUG : Instanciação da coo_matrix realizada em 0:00:00.001004 seconds
2023-12-2

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Módulo Online

In [52]:
from common import SimItem
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
#logging.basicConfig(
#    level=logging.DEBUG, format='%(asctime)s : %(levelname)s : %(message)s', datefmt="[%X]", handlers=[RichHandler(markup=True)]
#)

logger = logging.getLogger('Recomendador baseado em vizinhança item-item')

class NeighborhoodBasedRecs:

    def __init__(self, settings:Settings, ratings_df:pd.DataFrame, **kargs):
               
        self.settings = settings
        self.ratings_df = ratings_df
        
        self.neighborhood_size = kargs.get('neighborhood_size', 15)
        self.min_sim = kargs.get('min_sim', 0.0)
        self.max_candidates = kargs.get('max_candidates', 100)
        
    def recommend_items(self, user_id, num=6, **kargs):
        user_column = self.settings.user_column
        rating_column = self.settings.rating_column
        item_column = self.settings.item_column
        logger.debug(f'Buscando recomendações para o usuário: {user_id}')
        ratings = self.ratings_df
        # Aqui teremos uma lista onde cada row é um dicionário contendo os dados.
        active_user_items = ratings[ratings[user_column] == user_id].sort_values(by=rating_column, ascending=False).head(100)
        active_user_items = active_user_items.to_dict('records')
        #print(active_user_items)
        #return active_user_items
        if len(active_user_items) == 0:
            return {}
        
        # Criação do dicionário de itens e suas avaliações
        # Estes são os vinhos avaliados pelo usuário ativo
        item_ids = {item[item_column]: item[rating_column] for item in active_user_items}
        logger.debug(f'Itens do usuário ativo: {item_ids}')
        # Calculando a média das avaliações do usuário
        user_mean = sum(item_ids.values()) / len(item_ids)
        logger.debug(f'Média do usuário ativo: {user_mean}')
        conn = get_conn(self.settings)
        c = conn.cursor()
        # Perceba que estamos buscando na tabela de similaridade vinhos que o usuário ativo avaliou, 
        # avaliado por outros (source é um dos nossos vinhos)
        # mas target not in para excluir os vinhos já presentes nos itens que o usuário ativo já avaliou.
        # Encontre todos os itens similares aos itens avaliados pelo usuário (cuja similaridade seja maior que este parâmetro)
        query = """
        SELECT * FROM similarity
        WHERE source IN ({})
        AND target NOT IN ({})
        AND similarity > {}
        ORDER BY similarity DESC
        LIMIT {}
        """.format(','.join(map(lambda x: f'\'{str(x)}\'', item_ids.keys())), 
                ','.join(map(lambda x: f'\'{str(x)}\'', item_ids.keys())), 
                self.min_sim, 
                self.max_candidates)
        #print(query)
        candidate_items = c.execute(query).fetchall()
        logger.debug(f'Candidatos recuperados da tabela de similaridade: {candidate_items}')
        # Cada candidato é uma linha na tabela de Similaridade
        # O formato é ('datetime', source, target, similarity)
        recs = dict()
        # Para cada item similar a um dos itens similares do usuário...
        for candidate in candidate_items:
            # Selecionamos o vinho similar deste candidato
            target = candidate[SimItem.TARGET]
            prediction = 0
            sim_sum = 0.0
            # Aqui a vizinhança é selecinada. Perceba, quero os itens candidatos cujo target é o target selecionado. 
            # A partir do vinho target, olhamos para todas as similaridades presentes (respeitando o tamanho da vizinhança)
            # Isso nos dá uma list de itens que o usuário ativo avaliou.
            rated_items = [i for i in candidate_items if i[SimItem.TARGET] == target][:self.neighborhood_size]
            logger.debug(f"Itens avaliados pelo usuário pegos da vizinhança: {rated_items}")
            # Lembrando que deve ter mais de um único item nesta lista para que a recomendação por esta estratégia funcione...
            if len(rated_items) >= 1:
                # para cada item similar na lista de itens avaliados...
                for sim_item in rated_items:
                    # Vamos resgatar a rating do item similar e subtrair da média do usuário 
                    # (aqui estamos calculando a estimativa de avaliação dos itens)
                    # Primeira coisa é subtrair a média de avaliação do usuário pelo valor da similaridade
                    # Avaliação dada pelo usuário ativo no item j da vizinhança
                    r_u_j = float(item_ids[sim_item[SimItem.SOURCE]] - user_mean)
                    
                    sim_i_j = float(sim_item[SimItem.SIM])
                    prediction +=   sim_i_j * r_u_j
                    sim_sum += sim_i_j
                
                if sim_sum > 0:
                    recs[target] = {'predicao': float(user_mean) + prediction / sim_sum,
                                    'vizinhanca': [r[SimItem.SOURCE] for r in rated_items]}
        # Por fim retornamos até o máximo de num itens com as maiores predições de avaliação
        sorted_items = sorted(recs.items(), key=lambda item: -float(item[1]['predicao']))[:num]
        conn.close()
        return sorted_items


In [53]:
# test com a simple
ratings_df = pd.read_csv('./simple2.csv')
#logging.disable()
fc = NeighborhoodBasedRecs(settings_simple, ratings_df,neighborhood_size=5, min_sim=0.0)
reis = ['Severino', 'James', 'Tarantino', 'Helio', 'Pedro', 'Elias']
for rei in reis:
    print('--------------------------------------')
    print(f'Itens para o rei: {rei})')
    recommended_items = fc.recommend_items(rei)
    print(f'Itens recomendados: {recommended_items}')
# por que você acha que não recomendou nada? A vizinhança só tinha um item. 
# Troque o > por >= apenas como exercício na "if len(rated_items) > 1:"linha 78 do recomendador. 

2023-12-20 17:43:35,972 : DEBUG : Buscando recomendações para o usuário: Severino
2023-12-20 17:43:35,980 : DEBUG : Itens do usuário ativo: {'Cella': 5, 'Black Tower': 3, 'Reservado': 2, 'Gato Negro': 2, 'Toro': 2}
2023-12-20 17:43:35,982 : DEBUG : Média do usuário ativo: 2.8
2023-12-20 17:43:35,988 : DEBUG : Candidatos recuperados da tabela de similaridade: [('2023-12-20 17:30:26.704916', 'Cella', 'Alorna', 0.9999999999999999)]
2023-12-20 17:43:35,989 : DEBUG : Itens avaliados pelo usuário pegos da vizinhança: [('2023-12-20 17:30:26.704916', 'Cella', 'Alorna', 0.9999999999999999)]
2023-12-20 17:43:35,991 : DEBUG : Buscando recomendações para o usuário: James
2023-12-20 17:43:35,996 : DEBUG : Itens do usuário ativo: {'Cella': 4, 'Alorna': 4, 'Black Tower': 3, 'Gato Negro': 3, 'Toro': 3}
2023-12-20 17:43:35,997 : DEBUG : Média do usuário ativo: 3.4
2023-12-20 17:43:36,000 : DEBUG : Candidatos recuperados da tabela de similaridade: [('2023-12-20 17:30:26.704916', 'Black Tower', 'Reservad

2023-12-20 17:43:36,013 : DEBUG : Itens do usuário ativo: {'Cella': 5, 'Alorna': 5, 'Black Tower': 2, 'Reservado': 2, 'Gato Negro': 1, 'Toro': 1}
2023-12-20 17:43:36,014 : DEBUG : Média do usuário ativo: 2.6666666666666665
2023-12-20 17:43:36,017 : DEBUG : Candidatos recuperados da tabela de similaridade: []
2023-12-20 17:43:36,021 : DEBUG : Buscando recomendações para o usuário: Helio
2023-12-20 17:43:36,027 : DEBUG : Itens do usuário ativo: {'Black Tower': 5, 'Cella': 3, 'Alorna': 3, 'Gato Negro': 1, 'Toro': 1}
2023-12-20 17:43:36,028 : DEBUG : Média do usuário ativo: 2.6
2023-12-20 17:43:36,030 : DEBUG : Candidatos recuperados da tabela de similaridade: [('2023-12-20 17:30:26.704916', 'Black Tower', 'Reservado', 0.5794956538171452), ('2023-12-20 17:30:26.704916', 'Gato Negro', 'Reservado', 0.06901773983900074)]
2023-12-20 17:43:36,032 : DEBUG : Itens avaliados pelo usuário pegos da vizinhança: [('2023-12-20 17:30:26.704916', 'Black Tower', 'Reservado', 0.5794956538171452), ('2023-12

--------------------------------------
Itens para o rei: Severino)
Itens recomendados: [('Alorna', {'predicao': 5.0, 'vizinhanca': ['Cella']})]
--------------------------------------
Itens para o rei: James)
Itens recomendados: [('Reservado', {'predicao': 3.0, 'vizinhanca': ['Black Tower', 'Gato Negro']})]
--------------------------------------
Itens para o rei: Tarantino)
Itens recomendados: []
--------------------------------------
Itens para o rei: Helio)
Itens recomendados: [('Reservado', {'predicao': 4.5743018385486405, 'vizinhanca': ['Black Tower', 'Gato Negro']})]
--------------------------------------
Itens para o rei: Pedro)
Itens recomendados: []
--------------------------------------
Itens para o rei: Elias)
Itens recomendados: []


In [54]:
ratings_df = load_winex_ratings()
fc = NeighborhoodBasedRecs(settings_winex, ratings_df, max_candidates=6, neighborhood_size=100, min_sim=0.01)
users = [1221566, 1055317]
for user in users:
    recommended_items = fc.recommend_items(user)
    print(f'User: {user}. Itens recomendados: {recommended_items}')


2023-12-20 17:45:05,245 : DEBUG : Buscando recomendações para o usuário: 1221566
2023-12-20 17:45:05,251 : DEBUG : Itens do usuário ativo: {179855: 5.0, 111422: 4.5, 111417: 4.5, 179043: 4.5, 155438: 4.5, 111932: 4.0, 142875: 4.0, 111433: 4.0, 112381: 4.0, 138829: 4.0, 111475: 4.0, 111587: 4.0, 155368: 4.0, 167446: 3.5, 170982: 3.5, 155339: 3.5, 193580: 3.5, 111834: 3.5, 171266: 3.5, 135867: 3.5, 111740: 3.5, 139340: 3.5, 107023: 3.5, 180923: 3.5, 171256: 3.5, 155353: 3.5, 103435: 3.0, 145359: 3.0, 112094: 3.0, 162519: 3.0, 106629: 3.0, 174177: 2.5, 111755: 2.5, 111458: 2.5, 179027: 2.0}
2023-12-20 17:45:05,253 : DEBUG : Média do usuário ativo: 3.5714285714285716
2023-12-20 17:45:05,261 : DEBUG : Candidatos recuperados da tabela de similaridade: [('2023-12-20 17:30:26.704916', 179043, 179012, 0.19513173212181453), ('2023-12-20 17:30:26.704916', 179043, 179061, 0.1923626428404279), ('2023-12-20 17:30:26.704916', 179855, 180330, 0.14372044067355996), ('2023-12-20 17:30:26.704916', 111422

User: 1221566. Itens recomendados: [(180330, {'predicao': 5.0, 'vizinhanca': [179855]}), (179012, {'predicao': 4.5, 'vizinhanca': [179043]}), (179061, {'predicao': 4.5, 'vizinhanca': [179043]}), (111429, {'predicao': 4.5, 'vizinhanca': [111422]}), (111395, {'predicao': 4.5, 'vizinhanca': [111417, 111422]})]
User: 1055317. Itens recomendados: [(167429, {'predicao': 4.5, 'vizinhanca': [167433]}), (101677, {'predicao': 4.0, 'vizinhanca': [101678]}), (100067, {'predicao': 4.0, 'vizinhanca': [100092]}), (111503, {'predicao': 3.5, 'vizinhanca': [111398]}), (167592, {'predicao': 3.5, 'vizinhanca': [167450]}), (162611, {'predicao': 3.5, 'vizinhanca': [162505]})]
